## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [6]:
# requests 라이브러리 설치여부 확인
!pip show requests

Name: requests
Version: 2.32.5
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: c:\3.python\Python_WebScraping_Analysis\.venv\Lib\site-packages
Requires: certifi, charset_normalizer, idna, urllib3
Required-by: 


In [5]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.14.3
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: c:\3.python\Python_WebScraping_Analysis\.venv\Lib\site-packages
Requires: soupsieve, typing-extensions
Required-by: 


In [8]:
# reqeusts, bs4 import
import requests
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

In [9]:
# requests, bs4 버전 확인하기
print(f'requests 버전 = {requests.__version__}')
print(f'bs4 버전 = {bs4.__version__}')

requests 버전 = 2.32.5
bs4 버전 = 4.14.3


### 1. 뉴스 제목 추출하기

In [5]:
# IT/과학 뉴스 
req_param = {
    'sid': 105
}
# 
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보
req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
}

# requests 의 get() 함수 호출하기 

# 응답(response)이 OK 이면
res = requests.get(url, headers=req_header)
print(res.status_code)
print(res.ok)
print(type(res))
print(res.text)
# 응답 (response)에서 text 추출
# BeautifulSoup 객체 생성  
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")))
    # CSS 선택자
    a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")
    print(type(a_tags), type(a_tags[0]))
    for a_tag in a_tags:
        title = a_tag.text.strip()
        link = a_tag['href']
        print(title, link)
# print(soup.select("div.sa_text a[href*='mnews/article']"))

# <a> 태그 리스트 순회하기    
else:
    # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')  

https://news.naver.com/section/105
200
True
<class 'requests.models.Response'>
<!doctype html>
<html lang="ko" data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36">
	<head>
		<meta charset="utf-8">
		<meta http-equiv="X-UA-Compatible" content="IE=edge">
		<meta name="referrer" contents="always">
		<meta http-equiv="refresh" content="600">
		<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" />
		<meta property="og:title" content="IT/과학 : 네이버 뉴스">
		<meta property="og:type" content="website">
		<meta property="og:url" content="https://news.naver.com/section/105">
		<meta property="og:image" content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png">
		<meta property="og:description" content="모바일, 인터넷, SNS, 통신 등 IT/과학 분야 뉴스 제공">
		<meta property="og:article:author" content="네이버">
		<meta name="twitter:car

### 1.1 뉴스제목 추출하는 함수 선언하기

In [3]:
import requests
from bs4 import BeautifulSoup

#section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
section_dict = {'정치':100,'경제':101,'사회':102,'생활/문화':103,'세계':104,'IT/과학':105}

def print_news(section_name):  #print_new('생활/문화') 
    sid = section_dict.get(section_name,'정치')
    url = f'https://news.naver.com/section/{sid}'
    print(f'{section_name} 뉴스 {url}')
    req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
    }
    res = requests.get(url, headers=req_header)
    if res.ok:
        soup = BeautifulSoup(res.text,'html.parser')
        # CSS 선택자를 사용해서 a tag 목록 가져오기
        a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/']")
        # <a> 태그 리스트 순회하기    
        for a_tag in a_tags:
            title = a_tag.text.strip()
            link = a_tag['href']
            print(title, link)
    else:
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')

In [ ]:
print_news(103,section_dict[103])

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [1]:
import requests
import os

# 육아일기 73회차
req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=812354&no=208&week=sun',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/812354/208/20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_3.jpg'
]

for img_url in img_urls:
    # requests 의 get(url, headers) 함수 호출하기 
    res = requests.get(img_url, headers=req_header)
    print(res.status_code)        
    # binary 응답 데이터 가져오기
    img_data = res.content    
    # url에서 파일명만 추출하기
    file_name = os.path.basename(img_url)
    print(file_name)        
    # binday data를 file에 write하기
    with open(file_name,'wb') as file:
        print(f'Writing to {file_name}({len(img_data):,} bytes)')
        file.write(img_data)

200
20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_1.jpg
Writing to 20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_1.jpg(124,462 bytes)
200
20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_2.jpg
Writing to 20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_2.jpg(141,956 bytes)
200
20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_3.jpg
Writing to 20251223155029_0d14a9e6cdd6b23342f96d1bd33d01c3_IMAG01_3.jpg(170,913 bytes)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [ ]:
import requests

upload_files = {
    
}
#print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.



200


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [ ]:
# 캡차 키 발급 요청


In [ ]:
# 캡차 이미지 요청


In [ ]:
#  사용자 입력값 검증 요청



* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
# 사용자 입력값 검증 요청



Error Code: 403


### 5. 블로그 검색하기

In [ ]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': '',
    'X-Naver-Client-Secret': '',
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'


# requests get(url, params, headers) 요청 

# json() 함수로 응답 결과 가져오오기
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'

# 'data/nhnblog.txt' 파일 생성하기
